In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd, os
pd.set_option('display.max_colwidth', None)

data_test1 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/eng_laptop_dev_task1.jsonl', encoding='utf8'), lines=True)
data_test2 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/eng_laptop_dev_task2.jsonl', encoding='utf8'), lines=True)
data_test3 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/data/eng_laptop_dev_task3.jsonl', encoding='utf8'), lines=True)
data_test11 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/eng_restaurant_dev_task1.jsonl', encoding='utf8'), lines=True)
data_test21 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/eng_restaurant_dev_task2.jsonl', encoding='utf8'), lines=True)
data_test31 = pd.read_json(open('/kaggle/input/test-dimabsa-2026/data/eng_restaurant_dev_task3.jsonl', encoding='utf8'), lines=True)
data_test = pd.concat([data_test1, data_test2, data_test3, data_test11, data_test21, data_test31])

FILE_NAME = "eng_laptop_train_alltasks.jsonl"

file_name = os.path.join('/kaggle/input/test-dimabsa-2026/data/', FILE_NAME)
data = pd.read_json(open(file_name, encoding='utf8'), lines=True)


file_name = os.path.join('/kaggle/input/test-dimabsa-2026/data/', "eng_restaurant_train_alltasks.jsonl")
data_extra = pd.read_json(open(file_name, encoding='utf8'), lines=True)
data


,ID,Text,Quadruplet
0,laptop_quad_dev_1,"this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .","[{'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'pretty', 'VA': '7.12#7.12'}, {'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'stylish', 'VA': '7.12#7.12'}]"
1,laptop_quad_dev_2,for now i ' m okay with upping the experience & device to 3 out of 5 stars .,"[{'Aspect': 'device', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'NULL', 'VA': '5.50#5.25'}]"
2,laptop_quad_dev_3,"seems unlikely but whatever , i ' ll go with it .","[{'Aspect': 'NULL', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'NULL', 'VA': '5.00#5.12'}]"
3,laptop_quad_dev_4,this version has been my least favorite version i ' ve had for the following reasons listed bellow the pros .,"[{'Aspect': 'version', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'least favorite', 'VA': '3.30#6.60'}]"
4,laptop_quad_dev_5,- biggest disappointment is the track pad .,"[{'Aspect': 'track pad', 'Category': 'HARDWARE#GENERAL', 'Opinion': 'disappointment', 'VA': '2.50#6.00'}]"
...,...,...,...
4071,laptop_quad_train_2930,i think the sound could be better .,"[{'Aspect': 'sound', 'Category': 'MULTIMEDIA_DEVICES#OPERATION_PERFORMANCE', 'Opinion': 'could be better', 'VA': '4.17#5.50'}]"
4072,laptop_quad_train_2931,"i didn ' t want to spend much on this as it is my first laptop , but i was convinced by a few reviews to upgrade to this model as it has more ram and performs better in several areas .","[{'Aspect': 'performs', 'Category': 'LAPTOP#OPERATION_PERFORMANCE', 'Opinion': 'better', 'VA': '7.00#6.88'}]"
4073,laptop_quad_train_2932,"an awesome product , well built - well worth your time and money .","[{'Aspect': 'product', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'awesome', 'VA': '7.50#7.50'}, {'Aspect': 'product', 'Category': 'LAPTOP#QUALITY', 'Opinion': 'well built', 'VA': '6.83#7.00'}, {'Aspect': 'product', 'Category': 'LAPTOP#QUALITY', 'Opinion': 'well worth', 'VA': '7.00#7.17'}]"
4074,laptop_quad_train_2933,backlit keyboard is great ; feels sturdy ; fast processing .,"[{'Aspect': 'backlit keyboard', 'Category': 'KEYBOARD#GENERAL', 'Opinion': 'great', 'VA': '7.67#7.83'}, {'Aspect': 'NULL', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'sturdy', 'VA': '7.83#8.17'}, {'Aspect': 'NULL', 'Category': 'CPU#OPERATION_PERFORMANCE', 'Opinion': 'fast', 'VA': '8.00#8.33'}]"


## Prepare Seq2Seq Data

### Subtask:
Transform the loaded dataset into a seq2seq format. This involves converting each sentence's quadruplets into a target string of aspect-opinion pairs (e.g., 'aspect1#opinion1|aspect2#opinion2'), suitable for a seq2seq model. Handle cases where aspects or opinions are 'NULL'.


In [4]:
def process_quadruplets_to_target_string(quadruplets):
    target_parts = []
    for q in quadruplets:
        aspect = q.get('Aspect', 'NULL')
        opinion = q.get('Opinion', 'NULL')

        target_parts.append(f"{aspect}#{opinion}|")

    return ''.join(target_parts)

# Apply the function to create the 'TargetString' column
data['TargetString'] = data['Quadruplet'].apply(process_quadruplets_to_target_string)
data_extra['TargetString'] = data_extra['Quadruplet'].apply(process_quadruplets_to_target_string)

# Display the first few rows with the new 'TargetString' column
display(data[['Text', 'Quadruplet', 'TargetString']].head())

,Text,Quadruplet,TargetString
0,"this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .","[{'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'pretty', 'VA': '7.12#7.12'}, {'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'stylish', 'VA': '7.12#7.12'}]",unit#pretty|unit#stylish|
1,for now i ' m okay with upping the experience & device to 3 out of 5 stars .,"[{'Aspect': 'device', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'NULL', 'VA': '5.50#5.25'}]",device#NULL|
2,"seems unlikely but whatever , i ' ll go with it .","[{'Aspect': 'NULL', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'NULL', 'VA': '5.00#5.12'}]",NULL#NULL|
3,this version has been my least favorite version i ' ve had for the following reasons listed bellow the pros .,"[{'Aspect': 'version', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'least favorite', 'VA': '3.30#6.60'}]",version#least favorite|
4,- biggest disappointment is the track pad .,"[{'Aspect': 'track pad', 'Category': 'HARDWARE#GENERAL', 'Opinion': 'disappointment', 'VA': '2.50#6.00'}]",track pad#disappointment|


## Initialize Tokenizer and Model

### Subtask:
Load a pre-trained tokenizer (e.g., from T5 or BART) and an appropriate encoder-decoder model from the `transformers` library. Configure the tokenizer for both input text and target sequences.


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
# Choose a pre-trained model checkpoint
BATCH_SIZE = 4
EPOCHS=25
MODEL_NAME = 'model_t5_25'
MODEL_CHECKPOINT = f'/kaggle/working/{MODEL_NAME}' if os.path.exists(f'/kaggle/working/{MODEL_NAME}') else 't5-base'
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
print(f"Tokenizer '{MODEL_CHECKPOINT}' loaded successfully.")

# Initialize the Seq2Seq model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
print(f"Model '{MODEL_CHECKPOINT}' loaded successfully.")

Tokenizer '/kaggle/working/model_t5_25' loaded successfully.


2026-01-15 19:15:27.007083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768504527.177993      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768504527.225455      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768504527.636744      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768504527.636776      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768504527.636780      55 computation_placer.cc:177] computation placer alr

Model '/kaggle/working/model_t5_25' loaded successfully.


In [6]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['Text'], max_length=512, truncation=True, padding="max_length")
    # Setup the tokenizer for targets
    if 'TargetString' in examples.keys():
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples['TargetString'], max_length=512, truncation=True, padding="max_length")
    
        model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the tokenization function to the data. We'll convert the pandas DataFrame to a Hugging Face Dataset first.
from datasets import Dataset

hf_dataset = Dataset.from_pandas(data)
tokenized_data = hf_dataset.map(tokenize_function, batched=True)
hf_dataset_extra = Dataset.from_pandas(data_extra)
tokenized_data_extra = hf_dataset_extra.map(tokenize_function, batched=True)


hf_dataset_test = Dataset.from_pandas(data_test)
tokenized_data_test = hf_dataset_test.map(tokenize_function, batched=True)

from transformers import DataCollatorForSeq2Seq

# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

from torch.utils.data import DataLoader
from datasets import concatenate_datasets

# Split the tokenized_data into training and evaluation sets
train_test_split_dataset = tokenized_data.train_test_split(test_size=0.3, train_size=0.7, seed=42)

train_dataset = concatenate_datasets([train_test_split_dataset['train'], tokenized_data_extra])
eval_dataset = train_test_split_dataset['test']
test_dataset = tokenized_data_test

print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Create DataLoader instances
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator
)

test_dataloader = DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator
)

print(f"Train DataLoader created with batch size: {BATCH_SIZE}")
print(f"Evaluation DataLoader created with batch size: {BATCH_SIZE}")
print(f"Test DataLoader created with batch size: {BATCH_SIZE}")

Map:   0%|          | 0/4076 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Training dataset size: 5137
Evaluation dataset size: 1223
Test dataset size: 1200
Train DataLoader created with batch size: 4
Evaluation DataLoader created with batch size: 4
Test DataLoader created with batch size: 4


In [17]:
import pandas as pd, regex as re, numpy as np, polars as pl
def postprocess(dataframe, stats):
	s = 0
	for index, row in dataframe.iterrows():
		if any((quad['Aspect'] not in row['Text'] and quad['Aspect'] != 'NULL') or (quad['Opinion'] not in row['Text'] and quad['Opinion'] != 'NULL') for quad in row['Quadruplets']):
			p = True
		else:
			s += 1
			p = False
		if not p:
			continue
		for quad in row['Quadruplets']:
			if quad['Aspect'] not in row['Text'] and quad['Aspect'] != 'NULL':
				# Incomplete NULL tag
				if quad['Aspect'] in 'NULL':
					quad['Aspect'] = 'NULL'
				# Case sensitivity
				if quad['Aspect'].lower() in row['Text'].lower():
					matches = re.findall(re.escape(quad['Aspect']), row['Text'], flags=re.IGNORECASE)
					if matches:
						quad['Aspect'] = matches[0]
						stats['aspect_case'] += 1
			if quad['Opinion'] not in row['Text'] and quad['Opinion'] != 'NULL':
				# Incomplete NULL tag
				if quad['Aspect'] in 'NULL':
					quad['Aspect'] = 'NULL'
				else:
					if re.match('^not ', quad['Opinion']) and f"n\'t {quad['Opinion'][4:]}" in row['Text']:
						quad['Opinion'] = f'\'t {quad["Opinion"][4:]}'
						stats['opinion_negation'] += 1
					# Case sensitivity
					if quad['Opinion'].lower() in row['Text'].lower():
						matches = re.findall(re.escape(quad['Opinion']), row['Text'], flags=re.IGNORECASE)
						if matches:
							quad['Opinion'] = matches[0]
							stats['opinion_case'] += 1
	return dataframe

def raw_text_to_quads(text):
	ts = text.split('|')
	quads = []
	for t in ts:
		if len(t) > 0 and '#' in t:
				quads.append({'Aspect': t.split('#')[0], 'Opinion': t.split('#')[1]})
	return quads

def postprocess_raw(output, innputs, stats):
	quads = raw_text_to_quads(output)
	for quad, innput in zip(quads, innputs):
		if quad['Aspect'] not in innput and quad['Aspect'] != 'NULL':
			# Case sensitivity
			if quad['Aspect'].lower() in innput.lower():
				matches = re.findall(re.escape(quad['Aspect']), innput, flags=re.IGNORECASE)
				if matches:
					quad['Aspect'] = matches[0]
					stats['aspect_case'] += 1
			# Incomplete NULL tag
			elif quad['Aspect'] in 'NULL':
				quad['Aspect'] = 'NULL'
		if quad['Opinion'] not in innput and quad['Opinion'] != 'NULL':
			# Case sensitivity
			if quad['Opinion'].lower() in innput.lower():
				matches = re.findall(re.escape(quad['Opinion']), innput, flags=re.IGNORECASE)
				if matches:
					quad['Opinion'] = matches[0]
					stats['opinion_case'] += 1
			# Incomplete NULL tag
			elif quad['Aspect'] in 'NULL':
				quad['Aspect'] = 'NULL'
			if re.match('^not ', quad['Opinion']) and f"n\'t {quad['Opinion'][4:]}" in innput:
				quad['Opinion'] = f'\'t {quad["Opinion"][4:]}'
				stats['opinion_negation'] += 1

	out = process_quadruplets_to_target_string(quads)
	return out

def label_ids_to_text(label_ids, tokenizer):
	label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
	decoded_text = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
	return decoded_text

def compute_pair_accuracy(pred_texts, label_texts):
	correct = 0
	total = 0
	tot_pred = 1
	correct_pred = 0

	for pred, label in zip(pred_texts, label_texts):
		pred_pairs = raw_text_to_quads(pred)
		label_pairs = raw_text_to_quads(label)

		correct_now = 0
		for l in label_pairs:
			if len([x for x in pred_pairs if x['Aspect'] == l['Aspect'] and x['Opinion'] == l['Opinion']]):
				correct_now += 1

		total += len(label_pairs)
		correct += correct_now
		tot_pred += len(pred_pairs)
		correct_pred += correct_now

	return (correct / total, correct_pred / tot_pred)

def compute_pair_precision_recall(pred_texts, label_texts):
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    total_labels = 0  # Total labels in the ground truth

    # Loop through each prediction and ground truth pair
    for pred, label in zip(pred_texts, label_texts):
        pred_pairs = raw_text_to_quads(pred)  # Predicted aspect-opinion pairs
        label_pairs = raw_text_to_quads(label)  # Ground truth aspect-opinion pairs

        # Count true positives and false positives
        for l in label_pairs:
            if len([x for x in pred_pairs if x['Aspect'] == l['Aspect'] and x['Opinion'] == l['Opinion']]):
                tp += 1  # Correct prediction for this label pair
            else:
                fn += 1  # Missed prediction for this label pair

        # Count false positives
        for p in pred_pairs:
            if not any(l['Aspect'] == p['Aspect'] and l['Opinion'] == p['Opinion'] for l in label_pairs):
                fp += 1  # Incorrect prediction (false positive)

        total_labels += len(label_pairs)

    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

def compute_metrics(eval_preds):
	preds, labels = eval_preds
	innputs = [
		text for text, labelss in zip(eval_dataset["Text"], eval_dataset["labels"])
		if labelss in labels
	]

	if isinstance(preds, tuple):
		preds = preds[0]

	decoded_preds = label_ids_to_text(preds, tokenizer)
	decoded_labels = label_ids_to_text(labels, tokenizer)

	# ---- RAW METRIC ----
	raw_acc = compute_pair_precision_recall(
		decoded_preds,
		decoded_labels
	)

	# ---- POSTPROCESSED METRIC ----
	stats = {'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}
	post_preds = [postprocess_raw(p, innputs, stats) for p in decoded_preds]

	post_acc = compute_pair_precision_recall(
		post_preds,
		decoded_labels
	)
	f1_raw =  (2 * raw_acc[0] * raw_acc[1]) / (raw_acc[0] + raw_acc[1])
	f1_post = (2 * post_acc[0] * post_acc[1]) / (post_acc[0] + post_acc[1])

	return {
		"pair_accuracy_raw_precision": raw_acc[0],
		"pair_accuracy_raw_recall": raw_acc[1],
		"pair_accuracy_raw_f1": f1_raw,
		"pair_accuracy_post_precision": post_acc[0],
		"pair_accuracy_post_recall": post_acc[1],
		"pair_accuracy_post_f1": f1_post,
		"stats_postprocessing": stats,
	}

## Define Training Arguments and Trainer

### Subtask:
Set up training arguments including parameters like learning rate, number of epochs, batch size, and evaluation strategy. The `report_to` argument will be updated to exclude 'wandb'. Utilize the `Trainer` class from the `transformers` library for simplified training management.


In [7]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

# Define output directory
output_dir = f'./resultst{MODEL_NAME}'

# Define TrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='best',#'no'
    metric_for_best_model='pair_accuracy_raw_f1',
    save_total_limit=2,
    seed=42,  # Ensure consistent shuffling
    predict_with_generate=True,
    report_to=[], # Exclude wandb and other reporting services
    fp16=torch.cuda.is_available() # Enable FP16 if a GPU is available
)

# Instantiate the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print("TrainingArguments and Trainer initialized successfully.")
print("Training Arguments:", training_args)
print("Trainer instance:", trainer)

NameError: name 'compute_metrics' is not defined

## Training

In [8]:
print("Starting model training...")
trainer.train()

Starting model training...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pair Accuracy Raw Precision,Pair Accuracy Raw Recall,Pair Accuracy Raw F1,Pair Accuracy Post Precision,Pair Accuracy Post Recall,Pair Accuracy Post F1,Stats Postprocessing
1,0.015800,0.011360,0.507527,0.406663,0.451531,0.507527,0.406663,0.451531,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
2,0.011400,0.008073,0.592639,0.545663,0.568182,0.592639,0.545663,0.568182,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
3,0.009700,0.007382,0.614663,0.601953,0.608241,0.614663,0.601953,0.608241,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
4,0.007400,0.006696,0.655748,0.609420,0.631736,0.655748,0.609420,0.631736,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
5,0.005300,0.006596,0.666256,0.621482,0.643091,0.666256,0.621482,0.643091,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
6,0.005100,0.006520,0.657625,0.626651,0.641765,0.657625,0.626651,0.641765,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
7,0.004400,0.006333,0.677946,0.644457,0.660777,0.677946,0.644457,0.660777,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
8,0.004700,0.006236,0.681628,0.654222,0.667644,0.681628,0.654222,0.667644,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
9,0.002900,0.006678,0.672792,0.651924,0.662194,0.672792,0.651924,0.662194,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"
10,0.002600,0.006945,0.666085,0.657668,0.661850,0.666085,0.657668,0.661850,"{'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}"


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=16075, training_loss=0.09570862481696459, metrics={'train_runtime': 27493.1874, 'train_samples_per_second': 4.671, 'train_steps_per_second': 0.585, 'total_flos': 7.8205417095168e+16, 'train_loss': 0.09570862481696459, 'epoch': 25.0})

In [9]:
trainer.save_model(MODEL_NAME)

In [10]:
#!zip /kaggle/working/model.zip /kaggle/working/model

In [11]:
#rm -rf /content/results

## Eval

In [12]:
import numpy as np
from transformers import EvalPrediction

print("Evaluating model performance...")
# Make predictions on the evaluation dataset
# Added predict_with_generate=True to ensure proper token ID generation
predictions_output = trainer.predict(eval_dataset)

# Get the predicted token IDs (no argmax needed here as predict_with_generate handles it)
predicted_ids = predictions_output.predictions
label_ids = predictions_output.label_ids

# Decode the predicted token IDs
predicted_ids = np.where(predicted_ids != -100, predicted_ids, tokenizer.pad_token_id)
decoded_predictions = label_ids_to_text(predicted_ids, tokenizer)

decoded_labels = label_ids_to_text(label_ids, tokenizer)

# Print a few examples for visual inspection
print("\n--- Sample Predictions vs. True Labels ---")
for i in range(min(5, len(eval_dataset))): # Print up to 5 examples
    original_text = eval_dataset[i]['Text']
    true_target = decoded_labels[i]
    predicted_target = decoded_predictions[i]

    print(f"\nOriginal Text: {original_text}")
    print(f"True Target:   {true_target}")
    print(f"Predicted:     {predicted_target}")
    print("---")

print(compute_metrics((predicted_ids, label_ids)))
print("\nEvaluation complete.")

Evaluating model performance...



--- Sample Predictions vs. True Labels ---

Original Text: screen display is bright and has excellent graphics .
True Target:   screen display#bright|graphics#excellent|
Predicted:     screen display#bright|screen display#excellent|
---

Original Text: would definitely buy this again if i needed to do so .
True Target:   NULL#NULL|
Predicted:     NULL#NULL|
---

Original Text: this laptop ' s construction is cheap and flimsy , the battery is not removable and the back case is nearly impossible to take off without damaging it .
True Target:   laptop ' s construction#cheap|laptop ' s construction#flimsy|battery#not removable|back case#NULL|
Predicted:     laptop ' s construction#cheap|laptop ' s construction#f
---

Original Text: i ' ve found the touch screen is pretty handy .
True Target:   touch screen#pretty handy|
Predicted:     touch screen#pretty handy|
---

Original Text: up date per may 13 / 2018 about two months ago , the charger wont work .
True Target:   charger#NULL|
Predict

In [12]:
# Load best checkpoint
MODEL_CHECKPOINT_BEST = '/kaggle/working/resultstmodel_t5_25/checkpoint-15432'
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
print(f"Tokenizer '{MODEL_CHECKPOINT}' loaded successfully.")

# Initialize the Seq2Seq model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT_BEST)
print(f"Model '{MODEL_CHECKPOINT_BEST}' loaded successfully.")

Tokenizer '/kaggle/working/model_t5_25' loaded successfully.
Model '/kaggle/working/resultstmodel_t5_25/checkpoint-15432' loaded successfully.


In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_cached())

0
0


/tmp/ipykernel_55/1632861553.py:5: FutureWarning: `torch.cuda.memory_cached` has been renamed to `torch.cuda.memory_reserved`
  print(torch.cuda.memory_cached())


In [14]:
import numpy as np
model.eval()
model.to('cuda')
generated_ids = []
import gc, tqdm
step = 20
for i in tqdm.trange(0, len(test_dataset), step):
    generated_ids.append(model.generate(
        input_ids=torch.tensor(list(test_dataset['input_ids'][i:i+step]), device='cuda'),  # Tokenized source inputs for the encoder
        attention_mask=torch.tensor(list(test_dataset['attention_mask'][i:i+step]), device='cuda'),  # Tokenized source inputs for the encoder
        num_beams=4,  # Optionally, use beam search
    ).cpu())
    gc.collect()
    torch.cuda.empty_cache()
import torch

# Find the maximum sequence length across all tensors
max_seq_len = max([a.shape[1] for a in generated_ids])  # Use shape[1] for the sequence length

# Pad the sequences to the maximum sequence length
padded_predictions_test = [
    torch.cat([a, torch.full((a.shape[0], max_seq_len - a.shape[1]), tokenizer.pad_token_id)], dim=1) if a.shape[1] < max_seq_len else a
    for a in generated_ids
]
padded_predictions_test = np.vstack(padded_predictions_test)
print(padded_predictions_test.shape)

100%|██████████| 60/60 [02:41<00:00,  2.68s/it]


NameError: name 'np' is not defined

In [18]:
#predictions_output_test = trainer.predict(test_dataset)
predicted_ids_test = padded_predictions_test#predictions_output_test.predictions
import numpy as np
# Replace -100 with tokenizer.pad_token_id before decoding
# This is a safeguard if -100 somehow still appears, though predict_with_generate=True should prevent it
data_test['Task2OutRaw'] = label_ids_to_text(predicted_ids_test, tokenizer)
data_test['Quadruplets'] = data_test['Task2OutRaw'].apply(raw_text_to_quads)
if not os.path.exists(f'/kaggle/working/preds/{MODEL_NAME}'):
    os.mkdir(f'/kaggle/working/preds/{MODEL_NAME}')
with open(f'/kaggle/working/preds/{MODEL_NAME}/test_pred_nopostprocess.jsonl', 'w') as f:
    f.write(data_test.to_json(lines=True, orient='records'))
stats = {'aspect_case': 0, 'opinion_case': 0, 'opinion_negation': 0}
data_test = postprocess(data_test, stats)
display(data_test)
with open(f'/kaggle/working/preds/{MODEL_NAME}/test_pred.jsonl', 'w') as f:
    f.write(data_test.to_json(lines=True, orient='records'))

,ID,Text,Aspect,Task2OutRaw,Quadruplets
0,lap26_aspect_va_dev_1,The touchscreen works very well,[touchscreen],touchscreen#very well|,"[{'Aspect': 'touchscreen', 'Opinion': 'very well'}]"
1,lap26_aspect_va_dev_2,I am so disappointed in HP,[HP],HP#so disappointed|,"[{'Aspect': 'HP', 'Opinion': 'so disappointed'}]"
2,lap26_aspect_va_dev_3,The keyboard is big enough to use for real typing,[keyboard],keyboard#big|,"[{'Aspect': 'keyboard', 'Opinion': 'big'}]"
3,lap26_aspect_va_dev_4,I like the screen size,[screen size],screen size#like|,"[{'Aspect': 'screen size', 'Opinion': 'like'}]"
4,lap26_aspect_va_dev_5,Lenovo is my favorite brand of computer,[Lenovo],Lenovo#favorite|,"[{'Aspect': 'Lenovo', 'Opinion': 'favorite'}]"
...,...,...,...,...,...
195,rest26_asqp_dev_196,"The food was good , but not great",NaN,food#not great|food#good|,"[{'Aspect': 'food', 'Opinion': 'not great'}, {'Aspect': 'food', 'Opinion': 'good'}]"
196,rest26_asqp_dev_197,Their butter cookies are awesome -absolutely love them,NaN,butter cookies#awesome|butter cookies#love|,"[{'Aspect': 'butter cookies', 'Opinion': 'awesome'}, {'Aspect': 'butter cookies', 'Opinion': 'love'}]"
197,rest26_asqp_dev_198,The variety of boba tea that they have is also very good,NaN,boba tea#very good|,"[{'Aspect': 'boba tea', 'Opinion': 'very good'}]"
198,rest26_asqp_dev_199,The service was great the workers were really nice and patient with me haha indecisive,NaN,service#great|workers#really nice|workers#patient|,"[{'Aspect': 'service', 'Opinion': 'great'}, {'Aspect': 'workers', 'Opinion': 'really nice'}, {'Aspect': 'workers', 'Opinion': 'patient'}]"
